In [176]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

In [149]:
train_df=pd.read_csv("datas/train.csv",encoding="gb2312")
test_df=pd.read_csv("datas/testA.csv",encoding="gb2312")

In [148]:
def encoded(train_df):
    sex_encoded, sex_categories = train_df["性别"].factorize()
    train_df["性别"]=sex_encoded
    date_encoded, date_categories = train_df["体检日期"].factorize()
    train_df["体检日期"]=date_encoded
    return train_df

In [152]:
encoded(train_df)
median=train_df.mean()
tem_df=train_df.fillna(median)
corr_matrix=tem_df.corr()
relative_df=corr_matrix["血糖"]
impotantcolumns=relative_df[relative_df.values>0.1].index.tolist()

In [151]:
relative_df.sort_values(ascending=False)

血糖             1.000000
年龄             0.252862
甘油三酯           0.227922
血红蛋白           0.148011
低密度脂蛋白胆固醇      0.146213
红细胞平均血红蛋白浓度    0.146058
总胆固醇           0.144753
*碱性磷酸酶         0.136771
尿素             0.136593
红细胞计数          0.121187
红细胞压积          0.120455
*丙氨酸氨基转换酶      0.117554
*r-谷氨酰基转换酶     0.112472
*天门冬氨酸氨基转换酶    0.089652
肌酐             0.087528
白细胞计数          0.087135
红细胞平均血红蛋白量     0.067214
中性粒细胞%         0.046148
id             0.045757
*球蛋白           0.043474
血小板体积分布宽度      0.040734
*总蛋白           0.040091
嗜碱细胞%          0.024687
血小板平均体积        0.022887
尿酸             0.019934
乙肝表面抗原         0.013570
单核细胞%          0.004719
嗜酸细胞%          0.002343
白蛋白            0.002285
乙肝e抗原          0.000469
乙肝核心抗体         0.000460
红细胞平均体积       -0.006325
体检日期          -0.008980
白球比例          -0.012292
乙肝e抗体         -0.013443
乙肝表面抗体        -0.025626
淋巴细胞%         -0.052161
血小板比积         -0.064953
血小板计数         -0.072670
高密度脂蛋白胆固醇     -0.073361
红细胞体积分布宽度     -0.075451
性别            -0

In [153]:
impotantcolumns.remove("血糖")

In [154]:
impotantcolumns

['年龄',
 '*丙氨酸氨基转换酶',
 '*碱性磷酸酶',
 '*r-谷氨酰基转换酶',
 '甘油三酯',
 '总胆固醇',
 '低密度脂蛋白胆固醇',
 '尿素',
 '红细胞计数',
 '血红蛋白',
 '红细胞压积',
 '红细胞平均血红蛋白浓度']

In [155]:
new_train_df=train_df[impotantcolumns]
new_test_df=test_df[impotantcolumns]
df=pd.concat([new_train_df,new_test_df])

In [156]:
df.fillna(df.median(),inplace=True)

In [159]:
df

,年龄,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,甘油三酯,总胆固醇,低密度脂蛋白胆固醇,尿素,红细胞计数,血红蛋白,红细胞压积,红细胞平均血红蛋白浓度
0,41,23.100,99.59,20.230,1.31,4.43,2.65,5.87,5.21,166.1,0.479,347.0
1,41,36.250,67.21,79.000,2.81,4.06,2.63,5.26,5.21,156.0,0.456,342.0
2,46,15.230,63.69,38.170,0.99,4.13,2.01,4.77,4.76,148.8,0.438,340.0
3,22,10.590,74.08,20.220,1.06,6.89,5.04,4.23,4.29,137.0,0.403,340.0
4,48,14.780,75.79,22.720,0.97,5.37,3.65,4.87,5.15,106.0,0.354,299.0
5,74,22.590,81.23,23.350,2.45,6.65,4.28,6.11,4.64,141.3,0.413,343.0
6,31,25.530,109.03,65.420,1.51,5.60,3.82,6.14,6.03,172.0,0.504,341.0
7,55,40.030,88.49,25.150,1.32,5.92,4.20,6.31,4.90,158.0,0.448,353.0
8,39,39.170,102.91,67.090,2.07,5.50,3.60,4.98,4.94,163.0,0.482,338.0
9,35,15.790,78.21,25.750,1.25,3.98,2.02,6.61,4.81,144.0,0.437,330.0


In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X_train, y_train)


In [ ]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [181]:
train_feat=df[0:5642]


test_feat=df[5642:]

In [184]:
train_feat["血糖"]=pd.read_csv("datas/train.csv",encoding="gb2312")["血糖"]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [168]:
predictors=train_df.columns.tolist()

In [170]:
predictors.remove("血糖")

In [172]:
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

In [174]:
def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = mean_squared_error(label,pred)*0.5
    return ('mse',score,False)

In [187]:
print('开始CV 5折训练...')
scores = []

train_preds = np.zeros(train_feat.shape[0])
test_preds = np.zeros((test_feat.shape[0], 5))
kf = KFold(len(train_feat), n_folds = 5, shuffle=True, random_state=520)
for i, (train_index, test_index) in enumerate(kf):
    print('第{}次训练...'.format(i))
    train_feat1 = train_feat.iloc[train_index]
    train_feat2 = train_feat.iloc[test_index]
    lgb_train1 = lgb.Dataset(train_feat1[predictors], train_feat1['血糖'])
    lgb_train2 = lgb.Dataset(train_feat2[predictors], train_feat2['血糖'])
    gbm = lgb.train(params,
                    lgb_train1,
                    num_boost_round=3000,
                    valid_sets=lgb_train2,
                    verbose_eval=100,
                    feval=evalerror,
                    early_stopping_rounds=100)
    feat_imp = pd.Series(gbm.feature_importance(), index=predictors).sort_values(ascending=False)
    train_preds[test_index] += gbm.predict(train_feat2[predictors])
    test_preds[:,i] = gbm.predict(test_feat[predictors])
print('线下得分：    {}'.format(mean_squared_error(train_feat['血糖'],train_preds)*0.5))


开始CV 5折训练...
第0次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.09321	valid_0's mse: 1.5466
[200]	valid_0's l2: 3.00322	valid_0's mse: 1.50161
[300]	valid_0's l2: 2.96706	valid_0's mse: 1.48353
[400]	valid_0's l2: 2.95984	valid_0's mse: 1.47992
[500]	valid_0's l2: 2.96248	valid_0's mse: 1.48124
Early stopping, best iteration is:
[408]	valid_0's l2: 2.95962	valid_0's mse: 1.47981
第1次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.07586	valid_0's mse: 1.03793
[200]	valid_0's l2: 2.03183	valid_0's mse: 1.01591
[300]	valid_0's l2: 2.01487	valid_0's mse: 1.00743
[400]	valid_0's l2: 2.01265	valid_0's mse: 1.00632
Early stopping, best iteration is:
[397]	valid_0's l2: 2.01257	valid_0's mse: 1.00628
第2次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.65319	valid_0's mse: 0.826593
[200]	valid_0's l2: 1.61016	valid_0's mse: 0.805078
[300]	valid_0's l2: 1.60457	valid_0's

In [106]:
from sklearn import preprocessing
X = df.values
X = preprocessing.scale(X)

In [107]:
X.shape

(6642, 8)

In [108]:
x_train = X[0:5642]
x_test = X[5642:]

In [109]:
x_train.shape

(5642, 8)

In [110]:
y_train=train_df["血糖"].values

In [111]:
y_train

array([ 6.06,  5.39,  5.59, ...,  5.24,  6.37,  6.  ])

In [112]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [113]:
lin_reg=LinearRegression()
lin_reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [132]:
mse=cross_val_score(lin_reg,x_train,y_train,scoring="neg_mean_squared_error",cv=10)

In [133]:
np.mean(mse)

-2.0843930524730432

In [134]:
lin_reg.predict(x_test)

array([ 5.74739224,  5.99068241,  5.23684898,  6.33645334,  5.30227026,
        5.44304849,  5.28846522,  5.68436118,  6.06789972,  6.34478316,
        4.49536728,  6.00068628,  6.39126394,  5.64511035,  6.47571567,
        4.51861111,  5.4359675 ,  6.43674335,  5.10947224,  5.81287535,
        5.13650302,  5.73051013,  6.47511808,  6.04094091,  5.62606552,
        5.41952515,  5.20285065,  5.37694969,  5.4842451 ,  5.87757076,
        5.3272866 ,  5.46177207,  5.65919684,  6.87486881,  5.82225737,
        4.78463617,  5.80888094,  5.48987001,  6.0819352 ,  5.85276623,
        5.88424111,  5.78012766,  5.7441793 ,  5.48594445,  5.98090342,
        6.15527003,  5.65511487,  5.36486702,  4.5844189 ,  5.6838129 ,
        4.96218052,  4.95525774,  4.89735436,  4.95923614,  4.39825796,
        4.49679658,  5.03295275,  4.71636422,  4.5691221 ,  5.2681003 ,
        4.916935  ,  5.231667  ,  8.0240743 ,  5.25352926,  4.88664535,
        5.12958183,  4.98457745,  4.66898894,  6.15996651,  5.75

In [135]:
result=pd.DataFrame({'pred':lin_reg.predict(x_test).tolist()})

In [136]:
result.to_csv('submit.csv',index=False,header=None)